In [4]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup
from urllib.parse import urlparse, urljoin
import re

##dbs星展, sinopac永豐
bank_urls = {
            'dbs':'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/default.page', 
            'sinopac':'https://bank.sinopac.com/sinopacBT/personal/credit-card/introduction/list.html',
            'cathy': 'https://www.cathaybk.com.tw/cathaybk/personal/product/credit-card/cards/',
            'firstbank': 'https://card.firstbank.com.tw/sites/card/CreditCardList'
             }
credit_card_urls = {}
soup_documents = {k:None for k in list(bank_urls.keys())}

for bank, url in bank_urls.items():
    loader = RecursiveUrlLoader(url=url)
    docs = loader.load()
    if bank == 'dbs':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if (a['href'].startswith('/personal-zh/cards')) and (a['href'].endswith('hyperlink'))]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'sinopac':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if a['href'].startswith('./')]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'firstbank':
        def firstbank_extractor(html:str):
            soup_obj = Soup(html, 'html.parser')
            credit_card_feature = [re.sub(r'\n+', '\n', d.text) for d in soup_obj.find_all('div', {"class":[["card-single-features"]]})]
            txt = ''
            for features in  credit_card_feature:
                split = features.split('\n', 1)
                txt += f"信用卡卡名：{split[0].strip()}\n信用卡特色: {split[1]}"
            return txt
        all_html = RecursiveUrlLoader(bank_urls['firstbank'], extractor=firstbank_extractor).load()
        soup_documents[bank] = all_html

        divs = [Soup(doc.page_content, 'html.parser').find_all('div', {"class":'card-single'}) for doc in docs][0]
        links = []
        for d in divs:
            link_tags = d.find_all('a')
            if len(link_tags) == 1:
                detail_url = link_tags[0]['href']
                links.append(detail_url)
            else:
                for tag in link_tags:
                    if tag.text == '詳細內容':
                        links.append(tag['href'])
                    else:
                        continue
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'cathy':
        def parser(html):
            ## main page
            divs = Soup(html, 'html.parser').find_all('div', {'class':'cubre-m-compareCard -credit'})
            if divs:
                divs = [re.sub(r'\n+', '\n', i.text) for i in divs]
                txt = (' ').join(divs)
                txt = re.sub(r' +', ' ', txt)
                txt = [t.replace('\n立即申辦','', 1) for t in txt.split('詳細說明') if t.startswith('\n立即申辦')]
                txt = ('').join(txt).replace('\n \n', '\n')
            else:
                ## deeper page
                divs = Soup(html, 'html.parser').find_all('div', class_=["cubre-o-textContent", "cubre-m-colorBanner__title","cubre-m-iconEssay__title","cubre-m-horGraphic__title","cubre-m-remind__title","cubre-m-puzzle__title","cubre-a-kvTitle -card"])
                divs = [d.text for d in divs if '您將離開本行官網 前往外部網站' not in d.text]
                uni_divs = []
                for d in divs[:len(divs)-1]:
                    if d not in uni_divs:
                        uni_divs.append(d)
                txt = re.sub(r'\n+', '\n', ('\n').join(uni_divs))
                txt = re.sub(r' +', ' ', txt)
            return txt
            
        docs = RecursiveUrlLoader(url=url, extractor=parser).load()
        for d in docs:
            d.metadata['bank'] = '國泰銀行'
            d.metadata['credit card'] = d.metadata['title'].split('-', 1)[0].strip()
        docs = [d for d in docs if '停發' not in d.metadata['title']]
        soup_documents['cathy'] = docs
        

In [3]:
from langchain_community.document_loaders.url_selenium import SeleniumURLLoader
# documents = {}
# for bank, links in credit_card_urls.items():
#     documents[bank] = SeleniumURLLoader(urls=links).load()
        

In [5]:
def sinopac_extractor(html: str) -> str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'tab-box'})]
    divs_txt.insert(0, re.sub(r'\n\n+', '\n', soup.find('div', {'class':'info'}).text))
    div_set = []
    for txt in divs_txt:
        if txt not in div_set:
            div_set.append(txt)
    txt = re.sub(r'[\t\r\xa0]', '', ('\n').join(div_set))
    txt = re.sub(r'  +', ' ', txt)
    txt = re.sub(r'\n+', '\n', txt)
    return txt.strip()

def dbs_extractor(html:str)->str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'flpweb-legacy'})]
    if divs_txt:
        div_set = []
        for txt in divs_txt:
            if txt not in div_set:
                div_set.append(txt)
        all_txt = ('\n').join(div_set)
    else:
        txt = soup.text
        txt = [t.strip() for t in txt.split('\n') if( t != 'more..') and (len(txt.strip()) != 1)]
        all_txt = ('\n').join(txt)

    all_txt = all_txt.replace('個人網路銀行\nCard+ 信用卡數位服務\n企業網路銀行', '')
    all_txt = re.sub(r'[\t\r\xa0]', '', all_txt)
    all_txt = re.sub(r'\xa0', '', all_txt)
    all_txt = re.sub(r'  +', ' ', all_txt)
    all_txt = re.sub(r'\n+', '\n', all_txt)
    all_txt = all_txt.split('信用卡刷卡優惠更多星展信用卡刷卡優惠信息，詳情請見刷卡優惠說明。')[0]

    if '假字假字假字假字假字假字假字假字假字假字' in all_txt:
        all_txt = re.sub('假字假字假字+', '', all_txt)

    pattern = r"網銀登入\n[\s\S]*?選擇您的網站 ▼\n個人金融\n個人金融\n財富管理\n星展豐盛理財\n星展豐盛私人客戶\n企業金融\n中小企業銀行\n企業及機構銀行\n星展集團\n關於我們"
    # Replace the matched portion with an empty string
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'Loading...\n尊享至上\n[\s\S]*?\n權益手冊\n更多獨享優惠'
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'新戶好禮\n[\s\S]*?\n更多星展信用卡'
    all_txt = re.sub(pattern, '', all_txt)
    all_txt = all_txt.split('謹慎理財 信用至上')[0]
    return all_txt.strip()

def firstbank_extractor(html:str)->str:
    divs = Soup(html, 'html.parser').find_all('div', {"class":['carousel-item', 'content-body']})
    return ('').join([re.sub(r'\n+', '\n', d.text) for d in divs])

for bank, links in credit_card_urls.items():
    docs = []
    if bank == 'sinopac':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=sinopac_extractor).load()
            doc[0].metadata['bank'] = '永豐銀行'
            docs.extend(doc)
    elif bank == 'dbs':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=dbs_extractor).load()
            if doc:
                doc[0].metadata['bank'] = '星展銀行'
                docs.extend(doc)
    elif bank == 'firstbank':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=firstbank_extractor).load()
            doc[0].metadata['bank'] = '第一銀行'
            docs.extend(doc)
    if soup_documents[bank]:
        soup_documents[bank].extend(docs)
    else:
        soup_documents[bank] = docs

In [7]:
for k,v in soup_documents.items():
    print(f"Bank {k}: {len(v)}")

Bank dbs: 23
Bank sinopac: 46
Bank cathy: 18
Bank firstbank: 37


## Extract credit card name from DBS, Sinopac

In [38]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import string
import re

chinese_punctuation = "。！；：，、【】《》『』——"
translator = str.maketrans('', '', chinese_punctuation)

for k in list(soup_documents.keys()):
    if (k == 'sinopac'):
        for index, doc in enumerate(soup_documents[k]):
            credit_card = doc.page_content.split('\n')[0]
            # print(credit_card)
            soup_documents[k][index].metadata['credit card'] = credit_card
    elif k == 'dbs':
        for index, doc in enumerate(soup_documents[k]):
            credit_card = doc.page_content.split('\n')[0].split('|')[0].translate(translator)
            match = re.search(r'星展.*?卡', credit_card)
            if match:
                soup_documents[k][index].metadata['credit card'] = match.group()  # Output: 星展HAPPY GO聯名卡
            else:
                match = re.search(r'星展.*?卡', doc.metadata['description'])
                if match:
                    soup_documents[k][index].metadata['credit card'] = match.group()
                else:
                    pass
    elif k == 'firstbank':
        for index, doc in enumerate(soup_documents[k]):
            credit_card = doc.metadata['title'].strip()
            match = re.search(r'^(.*?)\s*-\s*信用卡', credit_card)
            if match:
                credit_card = match.group(1).split('_')[0]
                if credit_card.endswith('卡') or 'pay' in credit_card.lower():
                    soup_documents[k][index].metadata['credit card']  = credit_card
                else:
                    soup_documents[k][index].metadata['credit card']  = ''
            else:
                pass

In [60]:
import mlflow
from langchain.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=300,
                                        length_function=len,
                                        is_separator_regex=True,
                                        chunk_overlap=50,
                                        separators=['。'])
banks = list(bank_urls.keys())
chunks = splitter.split_documents(soup_documents[banks[0]])
for b in banks[1:]:
    chunks.extend(splitter.split_documents(soup_documents[b]))

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-base-zh-v1.5', model_kwargs={'device':'mps'})
collection_name = 'credit_card_chunk300'
qdrant_url = 'http://localhost:6333'
# vec_store = QdrantVectorStore.from_existing_collection(
#                               collection_name=collection_name,
#                               embedding = embedding,
#                               url = qdrant_url)

vec_store = QdrantVectorStore.from_documents(chunks,
                              collection_name=collection_name,
                              embedding = embedding,
                              force_recreate=True,
                              url = qdrant_url)



## Self-querying retriever

In [66]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The url of the credit card webpage",
        type="string",
    ),
    AttributeInfo(
        name="title",
        description="The title of the credit card webpage",
        type="integer",
    ),
    AttributeInfo(
        name="description",
        description="The description of the credit card webpage",
        type="string",
    ),
    AttributeInfo(
        name="language",
        description="The language of the credit card webpage",
        type="string"
    ),
    AttributeInfo(
        name="bank",
        description="The bank that hosts the credit card webpage",
        type="string"
    ),
    AttributeInfo(
        name="content_type",
        description="The content type for a webpage refers to the MIME type or media type specified in the HTTP header, which tells the browser how to interpret the content, such as text/html for HTML documents, application/json for JSON data, or image/jpeg for JPEG images.",
        type="string"
    ),
    AttributeInfo(
        name="credit card",
        description="The credit card name",
        type="string or list[string]"
    ),
]
document_content_description = "The content between HTML tags that is displayed on a credit card webpage, such as headings, paragraphs, links, and other textual elements"

self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vec_store,
    document_content_description,
    metadata_field_info,
    search_kwargs={'k': 7}
)

In [71]:
query = "星展eco永續極簡卡有什麼優惠？"
self_query_retriever.invoke(query)


[Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/eco-ez?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-hyperlink', 'content_type': 'text/html', 'title': '星展eco永續極簡卡 | 星展銀行（台灣）', 'description': '國內／國外一般消費無腦刷，筆筆回饋2%、eco消費享最優5%回饋', 'language': None, 'bank': '星展銀行', 'credit card': '星展eco永續極簡卡', '_id': '211f6981-741b-4c50-b913-40cd3f5fd7c6', '_collection_name': 'credit_card_chunk300'}, page_content='星展eco永續極簡卡\n國內／國外一般消費回饋1.2%無上限日本／韓國／泰國／新加坡 / 美洲地區 享最優5%回饋\n一般消費回饋1.2%權益適用期間：2025/01/01~2025/06/30國內／國外一般消費，筆筆回饋1.2%國外指定地區最高回饋5%權益適用期間：2025/01/01~2025/06/30日本／韓國／泰國／新加坡/ 美洲地區 享加碼最高5%（含原 1.2% 及加碼 3.8% 每期帳單上限 800 點）eco消費，最高享10%回饋權益適用期間：2025/01/01~2025/06/30星展支持的社會企業及中小企業／Tesla充電資費／Gogoro 電池資費享最高10%回饋（含原 1.2% +加碼 8.8% 每期帳單上限 300 點）\n 星展eco永續極簡卡現金紅利回饋活動注意事項星展eco永續極簡卡國內及國外一般消費 1.2% 現金紅利回饋活動注意事項：活動期間：2025／01／01 ~ 2025／06／30'),
 Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/eco-ez?pid=tw-pweb-personal-zh_cards

In [72]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder 
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

retriever = vec_store.as_retriever(search_kwargs={'k':8, 'score_threshold':0.5})
    
llm = ChatOllama(model='qwen2:7b-instruct', temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

reranker = HuggingFaceCrossEncoder(model_name='BAAI/bge-reranker-base')
compressor = CrossEncoderReranker(model=reranker, top_n=5)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever_from_llm)


In [73]:
from langchain.retrievers import EnsembleRetriever

try:
    ensemble_retriever = EnsembleRetriever(retrievers=[self_query_retriever, compression_retriever], weights=[0.6, 0.4])
    retrieved_docs = ensemble_retriever.invoke(query)
except:
    ensemble_retriever = compression_retriever
    retrieved_docs = ensemble_retriever.invoke(query)

retrieved_docs

[Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/eco-ez?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-hyperlink', 'content_type': 'text/html', 'title': '星展eco永續極簡卡 | 星展銀行（台灣）', 'description': '國內／國外一般消費無腦刷，筆筆回饋2%、eco消費享最優5%回饋', 'language': None, 'bank': '星展銀行', 'credit card': '星展eco永續極簡卡', '_id': 'ab833e5e-66d2-44b9-8366-1ea70a0b6c58', '_collection_name': 'credit_card_chunk300'}, page_content='。倘因活動到期後更新，權益手冊內容無法即時更新，以致於官網與權益手冊有不同，悉以本行官網公告內容為準。星展eco永續極簡卡eco消費加碼8.8%現金紅利回饋活動注意事項：持卡人使用本卡於星展支持的社會企業及中小企業消費、或綁定本卡於官方APP中繳納全額Tesla充電資費、Gogoro（含PBGN車主）電池資費，方可認列額外加碼。加碼回饋依每期帳單計算，eco消費8.8%加碼回饋上限為每期帳單300 點。星展基金會支持的社會企業及中小企業包含鮮乳坊、茶籽堂、鄰家鮮生nextdoorfresh、直接跟農夫買、玩轉學校、甘樂文創、禾乃川、馳綠、艾瑪絲、覺亞、EVOPURE+優織隆、綠藤、藍鵲茶等'),
 Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/eco-ez?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-hyperlink', 'content_type': 'text/html', 'title': '星展eco永續極簡卡 | 星展銀行（台灣）', 'description': '

In [74]:

from langchain_core.runnables import RunnablePassthrough
import opencc

from opik.integrations.langchain import OpikTracer
import os
import opik
# opik.configure(use_local=True)

# Create the Opik tracer
# opik_tracer = OpikTracer(tags=["langchain", "ollama"])
# os.environ["OPIK_PROJECT_NAME"] = "credit-card-helper"
converter = opencc.OpenCC('s2t.json')


model = ChatOllama(model='qwen2:7b-instruct', temperature=0)
# .with_config({"callbacks": [opik_tracer]})

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI question-answering assistant. Your task is to answer the question based on the provided documents. The documents are part of the text from the description of the credit cards.
    The documents are not all relevant to the question. Please filter and reply with an answer. No pre-amble or explanation. Please output text only, not in markdown format.
    
    Documents: 
    {context}

    Question:
    {question}

    Answer:""",
)

def format_docs(docs):
    return "\n\n".join([f"DOCUMENT {index}\nTitle: {d.metadata['title']}\nReleased bank: {d.metadata['bank']}\n{d.page_content}" for index, d in enumerate(docs)])


chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | QUERY_PROMPT
    | model
    | StrOutputParser()
)

# question = "國泰銀行有什麼信用卡？"
response = chain.invoke(query)
# compression_retriever.invoke(question)
print(converter.convert(response))

星展eco永續極簡卡提供以下幾項優惠：

1. **國外指定地區一般消費回饋**: 在日本、韓國、泰國、新加坡及美洲地區使用實體卡片或行動支付工具（如Apple Pay、Samsung Pay等）進行刷卡消費，並以非新臺幣為交易幣別且在帳單上列示國外交易手續費時，可享有最高5%的回饋。

2. **eco消費加碼**: 使用本卡於星展支持的社會企業及中小企業消費、或綁定本卡於官方APP中繳納全額Tesla充電資費、Gogoro（含PBGN車主）電池資費，可獲得額外回饋。特定情況下，eco消費可享有最高11.8%的回饋。

3. **年費優惠**: 活動期間為2025/01/01至2025/06/30，新申請者首年年費NT$2,400，第二年起年費NT$3,000。若正卡年度累計交易次數達12次、年度累計消費金額達NT$30,000或使用信用卡電子帳單換發後的首年，可免繳次年年費。

4. **加碼升級活動**: 星展存戶、豐盛理財客戶及豐盛私人客戶綁定指定行動支付工具（如Apple Pay、Samsung Pay等）進行消費時，可享有額外回饋。具體回饋率視消費地點和類型而異，最高可達14.8%。

以上優惠詳情請參閱星展銀行網站或星展回饋獎勵計畫。
